In [ ]:
import os
import glob
import re
import configparser
import pandas as pd
import numpy as np
import xlwings as xl
import xlsxwriter
import math




In [ ]:
def inifile(file):

    global ini_data

    fpath=file
    
    config = configparser.ConfigParser()
    config.read(fpath)

    data = []
    for section in config.sections():
        for key in config[section]:
            value = config[section][key]
            data.append([section, key, value])

    ini_data = pd.DataFrame(data, columns=['Freq', 'Test', 'Value'])
    # print(ini_data)

    # ini_data=ini_data.loc[(ini_data['Test']=='calibrationtarget') | (ini_data['Test']=='GoldenPhase'),:]
    ini_data = ini_data.loc[(ini_data['Test'] == 'calibrationtarget') | (ini_data['Test'] == 'goldenphase'), :]
    # print(ini_data)

    split_data=ini_data['Freq'].str.replace('Name','Freq').str.split('=',expand=True)

    ini_data = ini_data.drop(columns='Freq',axis=1)
    split_data = split_data.drop(columns=0,axis=1)

    ini_data=pd.concat([ini_data,split_data],axis=1)
    ini_data=ini_data.rename(columns={1:'Freq'})

    # ini_data['Value'] = ini_data['Value'].replace('#', np.nan)
    # ini_data['Value']=ini_data['Value'].astype(float)
    ini_data = ini_data[ini_data['Value'] != '#'].reset_index(drop=True)
    ini_data['Value'] = ini_data['Value'].astype(float)
    
    # print(ini_data.at[2,'Value'].dtype)

    print(ini_data)

In [ ]:
%run A601_Column_List.ipynb

In [ ]:


folder_path = os.path.dirname(fpath)


regex = re.compile(".*MicrophoneCharacterization.*")


writer = pd.ExcelWriter('New_ini_Target.xlsx', engine='xlsxwriter')


for value in column_list:
    search_pattern = os.path.join(folder_path, f"*{value}*")
    folders = glob.glob(search_pattern)
    if len(folders) > 0:
        for folder in folders:
            print(f"Found folder for value {value}: {folder}")
    
            files = glob.glob(os.path.join(folder, "*.ini"))
            if len(files) > 0:
                for file in files:
                 
                    match = regex.search(file)
                    if match:
                        inifile(file)
                        
                        sheetname=f'{value}'
                        ini_data.to_excel(writer, sheet_name=sheetname, index=False)
            else:       
                print(f"No files found in folder {folder}")
    else:
        print(f"No folders found for value {value}")

# 保存 Excel 文件
writer.save()

